<a href="https://colab.research.google.com/github/sarthakbiswas97/design-llm-apps-exercises/blob/main/Chapter_2_Design_LLM_apps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from newspaper import Article

url = 'https://timesofindia.indiatimes.com/india/save-your-lives-first-when-pakistan-army-commander-abandoned-post-during-operation-sindoor/articleshow/121364227.cms'
article = Article(url)
article.download()
article.parse()

print("Title:", article.title)
print("Text:", article.text)
# newspaper3k also has other useful attributes like article.authors, article.publish_date, etc.


Title: 'Save your lives first': When Pakistan army commander abandoned post during Operation Sindoor
Text: India's Team Against Terror: All-Party Delegation Shares Delhi's Message To The World | Op Sindoor

NEW DELHI: The Indian Army’s precision strikes during Operation Sindoor not only dismantled terror infrastructure across the Line of Control (LoC) but also triggered widespread panic within the Pakistan army, with intercepted communications revealing that senior officers abandoned their posts mid-conflict.According to Indian Army sources, a commander from Pakistan’s 75th Infantry Brigade, stationed near Muzaffarabad in Pakistan-occupied Kashmir (PoK), refused to return during the height of the offensive, news agency IANS reported.When junior officers contacted him about reopening operations, he chillingly responded: "The office will open later, save your lives first."Intercepted radio chatter further revealed the commander had taken refuge in a mosque."Our commander sahab escaped wi

C4 is an English language dataset, constructed by filtering out text from the raw dataset with less than 0.99 probability of being English according to langdetect. However, a lot of non-English data persists in this dataset. If you know a second language, then use the realnewslike subset of C4 to find instances in which text from that language appears. In what contexts do these non-English text fragments appear? Could an LLM learn these languages using these leftover fragments?

In [ ]:
! pip install --upgrade datasets
!pip install langdetect


  Using cached langdetect-1.0.9.tar.gz (981 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=1d3bdab204ad74e6e84bc3f181eee3aa42002f07ab377a82c00e6019baf6a258
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


In [ ]:
from datasets import load_dataset
from langdetect import detect_langs

articles_to_process = 100
confidence_threshold = 0.99

try:
  realnewslike = load_dataset("allenai/c4", "realnewslike", split="train", streaming=True)

  for i, news_text in enumerate(realnewslike):
    if i >= articles_to_process:
      break

    article_text = news_text.get("text")

    if not article_text or not article_text.strip():
      continue

    try:
      detected_lang_list = detect_langs(article_text)

      if not detected_lang_list:
        continue

      # Get the top language detection (most probable)
      top_detection = detected_lang_list[0]
      detected_lang_code = top_detection.lang  # e.g., 'en', 'es'
      detected_lang_prob = top_detection.prob  # e.g., 0.999


      # Condition 1: Detected language is English, but confidence is below the threshold
      is_low_confidence_english = (detected_lang_code == 'en' and detected_lang_prob < confidence_threshold)

      # Condition 2: Detected language is NOT English
      is_not_english = (detected_lang_code != 'en')

      if is_low_confidence_english or is_not_english:
        print("Found low confidence English or non-English article")
        print("------------------------------------")
        if is_low_confidence_english:
          print(f"Type: Low Confidence English")
        else: # is_not_english
          print(f"Type: Detected as Non-English")

      print(f"Detected Language: {detected_lang_code} (Probability: {detected_lang_prob:.4f})")
      print("Snippet (first 300 chars):")
      print(article_text[:300] + "...") # Print a snippet for context
      print("------------------------------------")

    except Exception as e:
      print("got error in language detection", e)

except Exception as e:
  print("Got some error: ",e)







Resolving data files:   0%|          | 0/1024 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/512 [00:00<?, ?it/s]

Detected Language: en (Probability: 1.0000)
Snippet (first 300 chars):
After the martyrdom of St. Boniface, Vergilius was made Bishop of Salzburg (766 or 767) and laboured successfully for the upbuilding of his diocese as well as for the spread of the Faith in neighbouring heathen countries, especially in Carinthia. He died at Salzburg, 27 November, 789. In 1233 he was...
------------------------------------
Detected Language: en (Probability: 1.0000)
Snippet (first 300 chars):
"Whoever gets him, they'll be getting a good one," David Montgomery said.
INDIANAPOLIS — Hakeem Butler has been surrounded by some of the best wide receivers on the planet this week at the NFL Scouting Combine.
It’s an experience that might humble some. But for Butler, it has only enhanced his confi...
------------------------------------
Detected Language: en (Probability: 1.0000)
Snippet (first 300 chars):
8i announced today it has launched a web player, the 8i Portal, for its volumetric 3D video of people in 

# New section

Create a quality classifier using [fasttext](https://fasttext.cc/docs/en/support.html). Your positive examples can be drawn from Wikipedia, and the negative examples can be randomly drawn from the [unclean version of C4](https://huggingface.co/datasets/allenai/c4). Once trained, feed documents from the realnewslike subset of C4 to this classifier. Is this classifier able to do a good job?

In [ ]:
!pip install --upgrade fasttext
%cd fastText
!pip install .
!pip install numpy==1.26.4


  Using cached fasttext-0.9.3-cp311-cp311-linux_x86_64.whl
  Attempting uninstall: fasttext
    Found existing installation: fasttext 0.9.2
    Uninstalling fasttext-0.9.2:
      Successfully uninstalled fasttext-0.9.2
/content/fastText
Processing /content/fastText
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for fasttext: filename=fasttext-0.9.2-cp311-cp311-linux_x86_64.whl size=4313419 sha256=1b765e8f3b9e48a1985970e9301b8fa39cb3fab0dba4e9e9bf37ee86be3065af
  Stored in directory: /tmp/pip-ephem-wheel-cache-3q0d4_bh/wheels/1e/bc/93/e4d717605b2816cf84bf6796e766c05fcc2fd4feb3f170fdf3
Successfully built fasttext
  Attempting uninstall: fasttext
    Found existing installation: fasttext 0.9.3
    Uninstalling fasttext-0.9.3:
      Successfully uninstalled fasttext-0.9.3


In [ ]:
# Getting and preparing data
!wget https://dl.fbaipublicfiles.com/fasttext/data/cooking.stackexchange.tar.gz && tar xvzf cooking.stackexchange.tar.gz
!head cooking.stackexchange.txt
!wc cooking.stackexchange.txt
!head -n 12404 cooking.stackexchange.txt > cooking.train
!tail -n 3000 cooking.stackexchange.txt > cooking.valid

--2025-05-27 12:18:27--  https://dl.fbaipublicfiles.com/fasttext/data/cooking.stackexchange.tar.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 3.171.22.13, 3.171.22.68, 3.171.22.118, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|3.171.22.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 457609 (447K) [application/x-tar]
Saving to: ‘cooking.stackexchange.tar.gz.1’

cooking.stackexchan 100%[===================>] 446.88K  --.-KB/s    in 0.05s   

2025-05-27 12:18:27 (9.15 MB/s) - ‘cooking.stackexchange.tar.gz.1’ saved [457609/457609]

cooking.stackexchange.id
cooking.stackexchange.txt
readme.txt
__label__sauce __label__cheese How much does potato starch affect a cheese sauce recipe?
__label__food-safety __label__acidity Dangerous pathogens capable of growing in acidic environments
__label__cast-iron __label__stove How do I cover up the white spots on my cast iron stove?
__label__restaurant Michelin Three Star Restaurant; but 

In [ ]:
# Step 4: Import and use fastText
import fasttext
import numpy as np # Import to check version

print(f"Using NumPy version: {np.__version__}") # Should show 1.26.4

# Train the model
model = fasttext.train_supervised(input="cooking.train")
model.save_model("model_cooking.bin")

# Make predictions
# Note: model.predict returns a tuple (labels, probabilities)
# We usually care about the labels for simple prediction.
predictions1 = model.predict("Which baking dish is best to bake a banana bread ?")
print(f"Prediction 1: {predictions1}")

predictions2 = model.predict("Why not put knives in the dishwasher?")
print(f"Prediction 2: {predictions2}")

# Test the model
test_results = model.test("cooking.valid")
print(f"Test results (N, P@1, R@1): {test_results}")

# To get precision and recall at k=5:
test_results_k5 = model.test("cooking.valid", k=5)
print(f"Test results (N, P@5, R@5): {test_results_k5}")


Using NumPy version: 1.26.4
Prediction 1: (('__label__baking',), array([0.07257967]))
Prediction 2: (('__label__food-safety',), array([0.07451777]))
Test results (N, P@1, R@1): (3000, 0.135, 0.05838258613233386)
Test results (N, P@5, R@5): (3000, 0.06606666666666666, 0.14285714285714285)
